# What I am Going to DO to get it working
- take the literally just 6_1 and see what is wrong with the colorlist

In [3]:
import numpy as np 
import pandas as pd 
import csv
from itertools import *
from fractions import Fraction

In [35]:
rrcm = []

for matrix in df["Row Reduced Coloring Matrix"].values.tolist():
    strings = matrix[2:-2].split("], [")
#        print(strings)
    list_of_lists = [ list(crossing.split(",")) for crossing in strings ]
#        print(list_of_lists)
    list_of_intlists = [ [int(n) for n in string] for string in list_of_lists ]
    rrcm.append(list_of_intlists)
    list_of_intlists = []
    list_of_lists = []
    strings = []
    #list_of_colorlists = ColourList( list_of_intlists ,3 )


In [47]:
pd.to_csv("pdcolorlists.csv",pd.DataFrame(colorlists))

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"[2, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
1,"[2, 1, 3, 2, 1, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
2,"[2, 1, 3, 3, 2, 1, 3, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
3,"[2, 3, 3, 2, 1, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
4,"[2, 3, 3, 1, 2, 3, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,"[2, 1, 3, 3, 1, 2, 3, 2, 3, 1, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
1080,"[2, 3, 3, 2, 2, 1, 2, 3, 1, 3, 3, 3, 1]","[2, 3, 1, 3, 1, 2, 2, 1, 1, 2, 3, 1, 3]","[1, 3, 1, 2, 3, 3, 1, 1, 2, 2, 3, 1, 1]","[3, 3, 1, 1, 2, 1, 3, 1, 3, 2, 3, 1, 2]",None,None,None,None,None,None,None,None,None
1081,"[1, 1, 1, 2, 2, 1, 1, 3, 3, 2, 2, 2, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None
1082,"[1, 1, 1, 2, 2, 1, 1, 3, 2, 2, 2, 3, 3, 1]",None,None,None,None,None,None,None,None,None,None,None,None


In [46]:
count = 0

for x in enumerate(colorlists):
    for y in x[1]:
        count += 1
    if count > 4:
        print(count)
        print(x)
    count = 0


13
(496, [[2, 3, 3, 2, 1, 2, 1, 3, 3, 3, 1, 2, 3, 3, 3, 3, 1], [3, 2, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 3, 2, 3, 1, 3], [2, 2, 1, 1, 2, 2, 2, 3, 1, 2, 2, 1, 3, 2, 3, 1, 1], [1, 2, 1, 3, 3, 1, 3, 3, 1, 2, 3, 3, 3, 2, 3, 1, 2], [2, 2, 3, 3, 3, 1, 3, 3, 1, 2, 2, 3, 3, 2, 1, 3, 3], [1, 2, 3, 2, 1, 3, 1, 3, 1, 2, 3, 2, 3, 2, 1, 3, 1], [3, 2, 3, 1, 2, 2, 2, 3, 1, 2, 1, 1, 3, 2, 1, 3, 2], [2, 1, 1, 2, 1, 1, 1, 3, 2, 1, 3, 2, 3, 1, 1, 1, 3], [1, 1, 1, 1, 2, 3, 2, 3, 2, 1, 1, 1, 3, 1, 1, 1, 1], [3, 1, 1, 3, 3, 2, 3, 3, 2, 1, 2, 3, 3, 1, 1, 1, 2], [2, 3, 2, 1, 2, 1, 2, 3, 3, 3, 1, 1, 3, 3, 1, 2, 3], [1, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 2, 1], [3, 3, 2, 2, 1, 2, 1, 3, 3, 3, 3, 2, 3, 3, 1, 2, 2]])
13
(570, [[3, 1, 2, 1, 2, 3, 1, 2, 2, 3, 1, 3, 3, 3, 2, 3, 3, 3, 1], [2, 3, 1, 1, 2, 1, 3, 1, 1, 1, 1, 3, 2, 3, 2, 1, 3, 1, 3], [2, 1, 3, 2, 1, 1, 1, 3, 3, 1, 2, 3, 2, 3, 1, 1, 3, 1, 1], [2, 2, 2, 3, 3, 1, 2, 2, 2, 1, 3, 3, 2, 3, 3, 1, 3, 1, 2], [2, 3, 1, 3, 3, 3, 3, 1, 1, 2, 3, 3, 1, 3, 3, 3, 1, 3,

In [8]:
df = pd.read_csv("./data/knotinfo_data_complete.csv",index_col=False)#. index_col = "name")

In [3]:
max(df["determinant"])

377

In [9]:
all_primes = [3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367,373]

In [6]:
#len(df[df["determinant"]%373==0])

In [10]:
primes_used = []
for p in all_primes:
    p_colorable_knots = df[df["determinant"]%p==0]
    if len(p_colorable_knots) != 0:
        primes_used.append(p)
        p_colorable_braid = p_colorable_knots[["name","braid_notation"]]
        p_colorable_braid.to_csv("./data/name_braid_"+str(p)+"colorable.csv",index=False)
        p_colorable_braid = []
    else:
        continue
    p_colorable_knots=[]

In [12]:
print(primes_used)
gauss_primes_used =[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 313, 353]

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 313, 353]


In [14]:
if primes_used == gauss_primes_used:
    print('yes')
else:
    print('no')

yes


In [11]:
three_colorable_knots = df[df["determinant"]%3==0]

In [12]:
three_colorable_gauss = three_colorable_knots[["name","gauss_notation"]]

In [13]:
three_colorable_gauss.to_csv("./data/name_gaussfromgauss_3colorable.csv",index=False)

In [18]:
five_colorable_braid = five_colorable_knots[["name","braid_notation"]]

In [20]:
five_colorable_braid.to_csv("name_braid_5colorable.csv",index=False)

# to-do 
- make a function that compares the gauss and braid DLNs
    - we are dealing with list of lists with [[this is the first coloring,this is a check of the first coloring],[ this is another coloring,checking another coloring with a different method]] 
    - first check to see if each individual one
    - get the gauss and braid DLN
    - switch every element to abs
    - if element in gauss code is in 

- write a function that determines whether a number is prime lol
- get a list of prime determinants of all knots
- get all fields for all gauss and braids

In [2]:
DLN_gauss = pd.read_csv("./data/name_DLNfromgauss_3colorable.csv")[1:]#. index_col = "name")
DLN_braid = pd.read_csv("./data/name_DLNfrombraid_3colorable.csv")[1:]

In [25]:
DLN_gauss_frac = []
knot_list = []

for knot in DLN_gauss["Dihedral Linking Number"]:#.values.tolist():
#     print(knot)
    strings = knot[2:-2].split("], [")
#     print(strings)
    for string in strings:
        if string == "'No Dihedral Linking Number Exists'":
            knot_list.append("No Dihedral Linking Number Exists")
        else:
            knot_list.append(abs(Fraction(string)))
    DLN_gauss_frac.append(knot_list)
    knot_list = []

# DLN_gauss_frac

In [26]:
DLN_braid_frac = []
knot_list = []

for knot in DLN_braid["Dihedral Linking Number"]:#.values.tolist():
#     print(knot)
    strings = knot[2:-2].split("], [")
#     print(strings)
    for string in strings:
        if string == "'No Dihedral Linking Number Exists'":
            knot_list.append("No Dihedral Linking Number Exists")
        else:
            knot_list.append(abs(Fraction(string)))
    DLN_braid_frac.append(knot_list)
    knot_list = []

#DLN_braid_frac

In [27]:
for i in range(len(DLN_braid_frac)): #you need to do if in and remove... :))))
    for j in range(len(DLN_braid_frac[i])):
        if DLN_braid_frac[i][j] in DLN_gauss_frac[i]:
            DLN_braid_frac[i][j] = "yes" #DLN_gauss_frac[i].pop()
        else:
            DLN_braid_frac[i][j] = "NOOoooooooOOOOoooo"

yes_list = DLN_braid_frac    

In [31]:
for y in yes_list:
    for z in y:
        if z == 'yes':
            continue
        else:
            print(y)